http://metodsnumericos.blogspot.com.co/2012/06/metodo-de-euler-hacia-delante-y-hacia.html
https://portal.camins.upc.edu/materials_guia/250133/2011/EDOs_2.pdf%3Bjsessionid=376136EBB9AE531040FA381BFAA0BD3F <--
http://relopezbriega.github.io/blog/2016/01/10/ecuaciones-diferenciales-con-python/



In [1]:
import numpy as np

To do

- predictor corrector


# Tarea # 1: (problema de Cauchy para ecuaciones diferenciales ordinarias)

## a) Resolver el problema de Cauchy para ecuaciones diferencial (9)

$$y''(x) + 5y'(x) + 6y(x) = 0$$

Con condiciones $y(0) = y_0 = 1$,  $y'(0) = -6$

- Reducir a un sistema de ecuaciones lineales de primera orden, aplíquese el método de Euler de segundo orden y encuéntrese los las relaciones de recurrencia 
    - a) Para un algoritmo explícito 
    - b) Implícito. 
    - c) Descríbase un algoritmo predictor-corrector en la base de estas relaciones.
    
Encuéntrese valores de la función y(x); 0 <=x <= 1 en n =100 nodos.


## Solución

Para reducir el sistema decimos que $y' = z$, y podemos escribir el problema (9)

$$z'+5z+6y=0$$  $y(0) =1$, $z(0) = -6$

Utilizando las formulas de recurrencia se encuentran los valores de la solución en el intervalo.

### Euler hacia adelante explícito

$y_{i+1} = y_i + h f(x_i,y_i)$

In [2]:
def sol_ed(x):
    return np.e**(-3*x)*(4.-3*np.e**x)

xsol =sol_ed(np.arange(0,1.01,0.01))
xsol[:5]

array([ 1.        ,  0.94118611,  0.88468982,  0.83043114,  0.77833271])

In [3]:
def euler_adelante(y0, z0, a, b, n):
    """Encuentra solucion de ecuación f(y,z) usando método de euler hacia adelante.
    y0: valor inicial en y
    z0: valor inicial en z , z = y'
    [a,b]: intervalo de x en el que hay que encontrar los valores de la solución y(x).
    n: numero de nodos"""
    #inicializo variables
    h = (b - a)/n
    yi, zi = y0, z0
    
    xvect = [a]
    yvect = [y0] # y0
    zvect = [z0] # y'0
    
    for i in np.arange(a+h,b+h,h):
        yprima = zi
        zprima = -5.*zi-6.*yi
        yi = yi + h*yprima
        zi = zi + h*zprima
        #agrego y's y z's 
        xvect.append(i)
        yvect.append(yi)
        zvect.append(zi)
        
    return xvect, yvect, zvect
    

In [4]:
x1, y1, z1 = euler_adelante(1.,-6., 0., 1., 100.)

print "x: ", x1[:5], "\n"
print "y: ", y1[:5]
print "y real: ", xsol[:5], "\n"
print "z: ", z1[:5]

euler_adelante =  ((np.array(y1) - np.array(xsol)) ** 2).mean() #error cuadrado
euler_adelante

x:  [0.0, 0.01, 0.02, 0.029999999999999999, 0.040000000000000001] 

y:  [1.0, 0.94, 0.8824, 0.827116, 0.77406676]
y real:  [ 1.          0.94118611  0.88468982  0.83043114  0.77833271] 

z:  [-6.0, -5.76, -5.5283999999999995, -5.304924, -5.08930476]


6.5718629428724212e-05

### Euler hacia atrás

$y_{i+1} = y_i + h f(x_{i+1},y_{i+1})$

In [5]:
def euler_atras(y0, z0, a, b, n):
    """Encuentra solucion de ecuación f(y,z) usando método de euler hacia atras.
    y0: valor inicial en y
    z0: valor inicial en z
    [a,b]: intervalo de x en el que hay que encontrar los valores de la solución y(x).
    n: numero de nodos"""
    #inicializo variables
    h = (b - a)/n
    yi, zi = y0, z0
    
    xvect = [a]
    yvect = [y0] # y0
    zvect = [z0] # y'0
    
    
    for i in np.arange(a+h,b+h,h):
        yprima = zi #f(xi,yi)
        zprima = -5.*zi-6.*yi # f'(xi,yi)
        
        yi_1 = yi + h*yprima # yi+1
        zi_1 = zi + h*zprima # zi+1
        yprima_ = zi_1 #f(xi+1,yi+1)
        zprima_ = -5.*zi_1-6.*yi_1 # f'(xi+1,yi+1)
        
        yi = yi + h*yprima_ # ' yi+1 se calcula con  z' y y' de xi+1 y yi+1
        zi = zi + h*zprima_ # ' zi+1 se calcula con  z' y y' de xi+1 y yi+1
        # agrego y's y z's 
        xvect.append(i)
        yvect.append(yi)
        zvect.append(zi)
        
    return xvect, yvect, zvect
    

In [6]:
x2, y2, z2 = euler_atras(1.,-6., 0., 1., 100)

print "x: ",x2[:5], "\n"
print "y: ",y2[:5]
print "y real: ",xsol[:5], "\n"
print "z: ",z2[:5]

euler_atras = ((np.array(y2) - np.array(xsol)) ** 2).mean() #error cuadrado
euler_atras

x:  [0.0, 0.01, 0.02, 0.029999999999999999, 0.040000000000000001] 

y:  [1.0, 0.9424, 0.88703476, 0.8338282999240001, 0.7827070653079877]
y real:  [ 1.          0.94118611  0.88468982  0.83043114  0.77833271] 

z:  [-6.0, -5.7684, -5.54465676, -5.3285197511639995, -5.119746164228679]


7.3120289540877634e-05

## Método Predictor corrector

Este método consta de un paso predictor que estima la solución  y un paso corrector que mejora la precisión en cada intervalo.


**Predictor**

$y'(x) = f(x,y)$, $y(x_0)=y_0$ 

$\tilde{y}_{i+1} = y_i + hf(x_i,y_i)$. Es Euler hacia adelante.


**Corrector**

$y_{i+1}=y_i+\frac{h(f(x_i,y_i)+f(x_{i+1},\tilde{y}_{i+1}))}{2}$  Es un trapezoide.

Hay variaciones de este método, se puede hacer el segundo paso varias veces, corregir hasta que el error sea menor que una tolerancia.


In [7]:
def predictor_corrector(y0, z0, a, b, n):
    """Encuentra solucion de ecuación f(y,z) usando método predictor corrector
    con predictor de euler hacia adelante.
    y0: valor inicial en y
    z0: valor inicial en z
    [a,b]: intervalo de x en el que hay que encontrar los valores de la solución y(x).
    n: numero de nodos"""
    #inicializo variables
    h = (b - a)/n
    yi, zi = y0, z0
    xvect = [a]
    yvect = [y0]
    zvect = [z0]
    
    for i in np.arange(a+h,b+h,h):
        # Predictor
        yprima = zi
        zprima = (-5.*zi)-(6.*yi)
        yi_ = yi + h*yprima  #euler hacia adelante
        zi_ = zi + h*zprima
        
        yprima_ = zi_
        zprima_ = (-5.*zi_)-(6.*yi_)
        
        # Corrector
        yi = yi +(h*(yprima+yprima_))/2. #y_i+1 = yi+ h()
        zi = zi + (h*(zprima+zprima_))/2.
        
        #agrego y's y z's 
        xvect.append(i)
        yvect.append(yi)
        zvect.append(zi)
        
    return xvect, yvect, zvect

In [8]:
x3, y3, z3 = predictor_corrector(y0=1.,z0=-6., a=0., b=1., n=100.)
#print "x: ",x3[:5], "\n"
print "y: ",y3[:8]
print "y real: ",xsol[:8], "\n"
#print "z: ",z3[:5]

pred_corr = ((np.array(y3) - np.array(xsol)) ** 2).mean() #error cuadrado
pred_corr

y:  [1.0, 0.9412, 0.8847166900000001, 0.8304701446405001, 0.7783830274063392, 0.7283805096307566, 0.6803901919818938, 0.6343420283566225]
y real:  [ 1.          0.94118611  0.88468982  0.83043114  0.77833271  0.72831965
  0.68031954  0.63426228] 



1.2310332083648734e-08

# b) Resolver el problema de Cauchy para sistemas de ecuaciones diferenciales 

- Describa un algoritmo que permita estimar valores de las funciones $\{y_1(t1), y_2(t1)\}$ usando las fórmulas de Runge-Kutta con la precisión $\varepsilon$ = 0.0001 utilizando el esquema de Runge para estimar el valor del error posible y disminuyendo sucesivamente el paso de la integración $h_0 = 0.1$, $h_{i+1}=h_i/2, i = 1,2,3,...$

$$\dot{x} = 3x+y$$  
$$\dot{y}=x+y-1$$

$x(0)=1/3$,  $y(0)=-1/3$

**Runge Kutta**

En los métodos de Runge-Kutta, el orden de precisión aumenta al utilizar puntos intermedios en cada intervalo. Una mayor precision implica además que los errores decrecen más rápido al reducir h, en comparaciOn con los métodos con precisión baja.

$$\frac{dx}{dt}= y_1(x,y,t) =3x +y$$

$$\frac{dy}{dt}= y_2(x,y,t) =x+y-1$$

$k_1=h y_1(x,y,t)$    

$k_2=h y_1(x+k_1/2,y+l_1/2,t+h/2)$

$k_3=h y_1(x+k_2/2,y+l_2/2,t+h/2)$  

$k_4=h y_1(x+k_3,y+l_3,t+h)$        
<br>
$l_1=h y_2(x,y,t)$

$l_2=h y_2(x+k_1/2,y+l_1/2,t+h/2)$

$l_3=h y_2(x+k_2/2,y+l_2/2,t+h/2)$

$l_4=h y_2(x+k_3,y+l_3,t+h)$

$$x(t+h)= x(t)+(k_1+2k_2+2k_3+k_4)/6$$
$$y(t+h)= y(t)+(l_1+2l_2+2l_3+k_4)/6$$

In [63]:
def f(x, y):
    result = 3.*x+y
    return result

def g(x, y):
     result = x+y-1.
    return result

def runge_kutta(x0, y0, t0, h, y1, y2, t):
    """
    Metodo de runge-kutta para resolver sistema de edo.
    
    t0: valor inicial
    x0: valor de x(t0), y0: valor de y(t0)
    
    retorna el valor de las funciones x y y en t."""
    
    x = x0
    y = y0
    
    xs = [x0]
    ys = [y0]
    
    for i in np.arange(t0, t+h, h):
        k1 = h*y1(x,y)
        l1 = h*y2(x,y)

        k2 = h*y1(x+k1/2.,y+l1/2.)
        l2 = h*y2(x+k1/2.,y+l1/2.)

        k3 = h*y1(x+k2/2.,y+l2/2.)
        l3 = h*y2(x+k2/2.,y+l2/2.)

        k4 = h*y1(x+k3,y+l3)
        l4 = h*y2(x+k3,y+l3)
        
        x = x+(k1+(2.*k2)+(2.*k3)+k4)/6.
        y = y+(l1+(2.*l2)+2.*l3+l4)/6.
        xs.append(x)
        ys.append(x)
        
    return xs[-1], ys[-1]


In [64]:
h = 0.1 # h0 inicial
tol = 0.0001
iters = 0

x,y= runge_kutta(x0 = 1./3, y0 = -1./3, t0 = 0, h = h, y1=f, y2=g, t=2)
x_,y_ = runge_kutta(x0 = 1./3, y0 = -1./3, t0 = 0, h = h/2., y1=f, y2=g, t=2)

while np.abs(x-x_)>tol and np.abs(y-y_) >tol:
    x,y= runge_kutta(x0 = 1./3, y0 = -1./3, t0 = 0, h = h, y1 = f, y2 = g, t=2)
    x_,y_ = runge_kutta(x0= 1./3, y0 = -1./3, t0 = 0, h = h/2., y1=f, y2=g, t=2)
    h = h/2.
    iters += 1
print "y1(t1): ",x,", y2(t1): ",y
print "h: ",h
print "iteraciones: ",iters
    

y1(t1):  60.2857550204 , y2(t1):  60.2857550204
h:  3.81469726563e-07
iteraciones:  18


# Tarea 2
## a) Encuentre la solución del problema de frontera con el paso h= 0.01 en cada caso.

Utilizando los métodos a) de disparos, b) diferencias finitos y c) de Numerov, encuéntrese la solución del problema de frontera con el paso h= 0.01 en cada caso.
(problema 9)

$$y''-\frac{y'}{2}+3y=2x^2$$

$y(1)+2y'(1)=0.6$

$y(1.3)=1$

**Método de colocación**

Es un problema de ecuación de segundo orden con una condición de frontera mixta.

$\alpha_0 = 1$, $\alpha_1 = 0$
$\beta_0 = 1$, $\beta_1 = 2$

Si elegimos la función de base 

$$\phi_0(x) = \delta + \gamma_0 x$$ y resolvemos el sistema

$1=\delta+1.3\gamma_0$
$0.6=\delta+3\gamma_0$ 

Obtenemos $\delta = \frac{111}{85}$, $\gamma_0 = \frac{-4}{17}$

In [14]:
from sympy import *

In [32]:
def func_base(d,Y):
    x = symbols('x')
    return d + Y*x


def met_colocacion(a,b,h,phi,alpha0,beta0,beta1,d,Y0):
    x = symbols('x')
    n = (b-a)/h
    Phi0 = func_base(d,Y0)
    Phi0_ = Phi0.diff(x)
    Phi0__ = Phi0_.diff(x)
    Ys = []#sumas van desde 1
    #ci = []
    Phis = []
    xeval = a
    
    cs = []
    
    p = -1./2
    q = 3.
    f = 2.*(x**2)
    
    xs = [a]
    
    for i in range(1,int(n)): #se tienen condiciones en los extremos 0 y n 
        xeval = xeval+h
        xs.append(xeval)
        Yi = -(beta0*((b-a)**2)+((i+1)*beta1*(b-a)))/(beta0*(b-a)+i*beta1)
        Phii = Yi*(x-a)**i + (x-a)**(i+1)
        Phii_ = Phii.diff(x)
        Phii__ = Phii_.diff(x)
        
        Ys.append(Yi)
        Phis.append([Phii,Phii_,Phii__])
        
        #aij bi cj i es nodo, j 
        aij = Phii__.evalf(subs={x:xeval}) + p.evalf(subs={x:xeval})*Phii_.evalf(subs={x:xeval}) + q.evalf(subs={x:xeval})*Phii.evalf(subs={x:xeval})
        bi = f.evalf(subs={x:xeval}) - Phi0__.evalf(subs={x:xeval}) - p.evalf(subs={x:xeval})*Phi0_.evalf(subs={x:xeval})- q.evalf(subs={x:xeval})*Phi0.evalf(subs={x:xeval})
        cj = bi/aij
        #y =phi0(xi)+sum(ci*phi(xi))
        
    
    
    return Phis,Ys,xs
            
#yn = Phi0.evalf(subs={x:xeval})+

In [33]:
P, Y, xs = met_colocacion(a=1.,b=1.3,h=0.01,phi=func_base,alpha0=1,beta0=1,beta1=2,d=111./85,Y0=-4./17)

In [35]:
print len(P)
#print xs

29


In [24]:
P, Y = met_colocacion(a=1.,b=2.,h=0.5,phi=func_base,alpha0=1,beta0=3,beta1=1,d=13./8,Y0=-5./8)#ejemplo profe

print P

print Y

In [ ]:
def func_base(d,Y):
    x = symbols('x')
    return d + Y*x


def met_colocacion(a,b,h,phi,alpha0,beta0,beta1,d,Y0):
    n = (b-a)/h
    Phi0 = func_base(d,Y0)
    Ys = []#sumas van desde 1
    Phis = []
    xeval = a
    x = symbols('x')
    xs = [a]
    for i in range(1,int(n)): #se tienen condiciones en los extremos 0 y n 
        xeval = xeval+h
        xs.append(xeval)
        Yi = -(beta0*((b-a)**2)+((i+1)*beta1*(b-a)))/(beta0*(b-a)+i*beta1)
        Phii = Yi*(x-a)**i + (x-a)**(i+1)
        Ys.append(Yi)
        Phis.append(Phii)
        yn =
    return Phis,Ys,xs
            
#yn = Phi0.evalf(subs={x:xeval})+
#http://docs.sympy.org/dev/modules/solvers/solveset.html#sympy.solvers.solveset.linsolve